In [53]:
from datetime import datetime
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from tqdm import tqdm
import time
# Custom API built off of Prop-Odds and Underdog API
from OddsAPI import API
api = API()

## prop-odds API Key
api.setAPI('Mw7lCnTZojOby32tBiqxxgCIEsnusFbsJfD7iosMU')

In [54]:
## display 100 rows|
pd.set_option('display.max_rows', 100)

In [55]:
## pivot: converts a dataframe that contains two seperate rows for each player (one for over, one for under) to have one row for each player
def pivot(df):
    pivot_df = pd.pivot_table(df, values='odds', index=['handicap', 'participant_name'], columns='name').reset_index()
    pivot_df.columns = ['line', 'participant_name', 'over_odds', 'under_odds']
    return pivot_df

## calculate odds
## adapted from Ammar Sulmanjee
def calculate_odds(x, y=None):
    if not y:
        if x>= 0: return 1 + x/100
        return 1 + 100/abs(x)
    
    ## calculate total odds
    imp_prob1 = (1 / calculate_odds(x)) * 100
    imp_prob2 = (1 / calculate_odds(y)) * 100

    ## remove VIG
    total_implied_prob = round(imp_prob1 + imp_prob2, 4)
    fair_prob1 = round(imp_prob1 / total_implied_prob * 100, 2)
    fair_prob2 = round(imp_prob2 / total_implied_prob * 100, 2)

    return [fair_prob1, fair_prob2]

## saves best bets
def download_bets(best_bets):
    ## format (take out uneccesary parts of market string)
    best_bets['market'] = best_bets['market'].str.replace(r'_|player|over_under', '', regex=True).str.strip()
    best_bets = best_bets.drop(columns=['game'])
    best_bets.to_csv(f"Bets_{datetime.now().strftime('%Y-%m-%d')}.csv", index=None)

## check palyer's raw data (debugging function)
def check_player(raw_bet_data, name, market, line=None):
    if line:
        display(raw_bet_data[(raw_bet_data['participant_name'] == name) & (raw_bet_data['market'] == market) & (raw_bet_data['handicap'] == line)])
        return
    display(raw_bet_data[(raw_bet_data['participant_name'] == name) & (raw_bet_data['market'] == market)])

def check_participants(df):
    # Count occurrences of each participant_name
    counts = df['participant_name'].value_counts()

    # Filter participants whose name appears twice
    participants_twice = counts[counts == 2].index
    df_filtered = df[df['participant_name'].isin(participants_twice)]

    return df_filtered


In [56]:
### Pull Underdog Lines
ud = api.get_fantasy_lines()

## get markets and games for the lines
markets = set(ud['market'])

games = api.get_nba_games()
games = [i['game_id'] for i in games['games']]

In [57]:
## save formatted best bets, and raw data to for check_player() function
bets = pd.DataFrame()
raw_bet_data = pd.DataFrame()


## run though each game and each market, tqdm is for progress bar
for game in tqdm(games, desc="Processing games"):
    for market in markets:
            
        # get sportsbook odds
        try:
            bookies_data = api.get_most_recent_odds(game, market)
        except Exception as e:
            if str(e) == "ERROR 422": continue
            else: 
                print(e)
                continue
             
        # format each books data     
        books = []
        for i in range(len(bookies_data['sportsbooks'])):
            df = pd.DataFrame.from_dict(bookies_data['sportsbooks'][i]['market']['outcomes'])
            df = df[df['timestamp'].str.contains(datetime.now().strftime('%Y-%m-%d'))]

            df['book'] = bookies_data['sportsbooks'][i]['bookie_key']
            df['market'] = market
            df['name'] = df['name'].apply(lambda x: "over" if "over" in x.lower() else "under")

            books.append(df)

        # add to raw_data for checking
        if len(books) > 0:
            raw_data = pd.concat(books)
            raw_bet_data = pd.concat([raw_bet_data, raw_data])

        # for each book, only get lines that match with under dog.
        # then conat all of these lines + odds
        new_datasets = []
        for dataset in books:
            dataset = pd.merge(dataset[['handicap', 'odds', 'participant_name', 'name', 'market']], ud, how='inner', on = ['participant_name', 'handicap' , 'market'])
            dataset = dataset[['handicap', 'odds', 'participant_name', 'name']]
            dataset = check_participants(dataset)
            
            if dataset.shape[0] > 0 and dataset.shape[1] > 0:
                # converts 2 rows (one over, one under) into 1 column for each player
                dataset = pivot(dataset)
                new_datasets.append(dataset)
        if new_datasets == []: continue

        ## concat all of these bets together
        final = pd.concat(new_datasets, axis=0)
        final = final.dropna()

        ## calculate median odds
        final[['over_odds', 'under_odds']] = final.apply(lambda row: pd.Series(calculate_odds(row['over_odds'], row['under_odds'])), axis=1)
        final = final.groupby(['participant_name', 'line']).agg({'over_odds':'median', 'under_odds':'median'}).reset_index()

        ## find ev, add market and game for each player
        final['ev'] = abs(final['over_odds'] - 50)
        final['market'] = market
        final['game'] = game


        if final.shape[0] > 0: 
            bets = pd.concat([bets, final[['participant_name', 'ev', 'market', 'line', 'over_odds', 'under_odds', 'game']]])
            # if bets[bets['ev'] > 4.5].shape[0] > 0: display(bets[bets['ev'] > 4.5].sort_values(by=['ev'], ascending=False))

Processing games: 100%|██████████| 4/4 [00:30<00:00,  7.71s/it]


In [58]:
bets = bets.sort_values(by=['ev'], ascending=False)
bets = bets.dropna()
bets = bets.drop_duplicates(keep='first', ignore_index=True)
bets = bets[['participant_name', 'market', 'line', 'over_odds', 'under_odds', 'ev', 'game']]
best_bets = bets[bets['ev'] > 4.5]

print(f"Total Bets found!: {bets.shape[0]}")
print(f"Total Positive EV Bets found!: {best_bets.shape[0]}")

Total Bets found!: 227
Total Positive EV Bets found!: 1


In [59]:
bets

,participant_name,market,line,over_odds,under_odds,ev,game
0,Jalen Smith,player_assists_points_rebounds_over_under,12.5,54.675,45.325,4.675,0314491e9364e4d7161dee189d893e42
1,Josh Hart,player_assists_points_rebounds_over_under,25.5,54.480,45.520,4.480,0314491e9364e4d7161dee189d893e42
2,Santi Aldama,player_assists_points_rebounds_over_under,23.5,45.560,54.440,4.440,4df8db26b532f4ee7725fd31840a51a1
3,Christian Wood,player_points_rebounds_over_under,17.5,53.770,46.230,3.770,bb46ad415c1b79dc1a1ae41ca77150d2
4,Jayson Tatum,player_blocks_over_under,0.5,46.255,53.745,3.745,bb46ad415c1b79dc1a1ae41ca77150d2
...,...,...,...,...,...,...,...
222,Isaac Okoro,player_threes_over_under,0.5,50.000,50.000,0.000,4df8db26b532f4ee7725fd31840a51a1
223,Jalen Brunson,player_threes_over_under,2.5,50.000,50.000,0.000,0314491e9364e4d7161dee189d893e42
224,Jalen Brunson,player_assists_points_over_under,40.5,50.000,50.000,0.000,0314491e9364e4d7161dee189d893e42
225,Myles Turner,player_assists_points_over_under,17.5,50.000,50.000,0.000,0314491e9364e4d7161dee189d893e42


In [60]:
check_player(raw_bet_data, 'Nic Claxton', 'player_points_over_under', 14.5)

,timestamp,handicap,odds,participant,participant_name,name,description,book,market


In [61]:
print(bets['market'].unique())

['player_assists_points_rebounds_over_under'
 'player_points_rebounds_over_under' 'player_blocks_over_under'
 'player_points_over_under' 'player_threes_over_under'
 'player_assists_rebounds_over_under' 'player_rebounds_over_under'
 'player_assists_points_over_under' 'player_assists_over_under'
 'player_turnovers_over_under' 'player_steals_over_under'
 'player_blocks_steals_over_under']


In [62]:
# dict to change market names for readability
markets_dict = {
    'player_assists_points_rebounds_over_under' : "Points + Rebounds + Assists",
    'player_points_over_under' : "Points",
    'player_assists_over_under' : "Assists",
    'player_threes_over_under' : 'Made Threes',
    'player_assists_rebounds_over_under': 'Assists + Rebounds',
    'player_rebounds_over_under': 'Rebounds',
    'player_assists_points_over_under' : 'Points + Assists',
    'player_points_rebounds_over_under':'Points + Rebounds',
    'player_blocks_steals_over_under' : 'Blocks + Steals',
    'player_turnovers_over_under' : 'Turnovers',
    'player_blocks_over_under' : 'Blocks',
    'player_steals_over_under' : 'Steals'

}

# replace values in market column w/ markets_dict
bets['market'].replace(markets_dict, inplace = True)

In [63]:
# this is for data visualization, has to be the last thing you do
# .style converts to style object, removing a lot of the attributes of df
'''
def highlight_values(val):
    # Change condition as needed
    if val >= 5.0:
        return 'background-color: green'
    elif val >= 3 and val < 5.0:
        return 'background-color: peru'
    else:
        return 'background-color: firebrick'

# Apply style to ev column
bets = bets.style.applymap(highlight_values, subset=['ev'])
best_bets = best_bets.style.applymap(highlight_values, subset=['ev'])
'''

"\ndef highlight_values(val):\n    # Change condition as needed\n    if val >= 5.0:\n        return 'background-color: green'\n    elif val >= 3 and val < 5.0:\n        return 'background-color: peru'\n    else:\n        return 'background-color: firebrick'\n\n# Apply style to ev column\nbets = bets.style.applymap(highlight_values, subset=['ev'])\nbest_bets = best_bets.style.applymap(highlight_values, subset=['ev'])\n"

In [64]:
# highlighting dataframe doesn't work when I change index to player name??
# answer: style object doesn't have abilities as normal df

# set index as player name
# best_bets.set_index('participant_name', inplace = True)
# bets.set_index('participant_name', inplace = True)

# bring back 0,1,...N index
# best_bets.reset_index(inplace=True)
# bets.reset_index(inplace=True)

In [65]:
bets

,participant_name,market,line,over_odds,under_odds,ev,game
0,Jalen Smith,Points + Rebounds + Assists,12.5,54.675,45.325,4.675,0314491e9364e4d7161dee189d893e42
1,Josh Hart,Points + Rebounds + Assists,25.5,54.480,45.520,4.480,0314491e9364e4d7161dee189d893e42
2,Santi Aldama,Points + Rebounds + Assists,23.5,45.560,54.440,4.440,4df8db26b532f4ee7725fd31840a51a1
3,Christian Wood,Points + Rebounds,17.5,53.770,46.230,3.770,bb46ad415c1b79dc1a1ae41ca77150d2
4,Jayson Tatum,Blocks,0.5,46.255,53.745,3.745,bb46ad415c1b79dc1a1ae41ca77150d2
...,...,...,...,...,...,...,...
222,Isaac Okoro,Made Threes,0.5,50.000,50.000,0.000,4df8db26b532f4ee7725fd31840a51a1
223,Jalen Brunson,Made Threes,2.5,50.000,50.000,0.000,0314491e9364e4d7161dee189d893e42
224,Jalen Brunson,Points + Assists,40.5,50.000,50.000,0.000,0314491e9364e4d7161dee189d893e42
225,Myles Turner,Points + Assists,17.5,50.000,50.000,0.000,0314491e9364e4d7161dee189d893e42
